In [1]:
import numpy as np
import xgboost as xgb
from wideboost.wrappers import wxgb

import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

(ds_train, ds_test), ds_info = tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train.batch(60000):
    a = i
    break
    
for i in ds_test.batch(60000):
    b = i
    break

In [2]:
xtrain = a[0].numpy().reshape([-1,28*28])
ytrain = a[1].numpy()

xtest = b[0].numpy().reshape([-1,28*28])
ytest = b[1].numpy()

dtrain = xgb.DMatrix(xtrain,label=ytrain)
dtest = xgb.DMatrix(xtest,label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    wbst = wxgb.train(param, dtrain, 20,watchlist,evals_result=ed1_results)
    
    output = min(ed1_results['test']['merror'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['R','I','Rn','In']),
    'extra_dims': hp.choice('extra_dims',range(16)),
    'objective': hp.choice('objective',['multi:softmax']),
    'eval_metric':hp.choice('eval_metric',[['merror']]),
    'num_class':hp.choice('num_class',[10]),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.choice('max_depth',range(1,11)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'alpha': 0, 'btype': 'R', 'colsample_bylevel': 0.9289424618466897, 'colsample_bytree': 0.9894516417124818, 'eta': 0.01942646236161944, 'eval_metric': ('merror',), 'extra_dims': 12, 'gamma': 0, 'lambda': 0, 'max_depth': 8, 'min_child_weight': 1.404308579819863e-07, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.5135951095107127}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `obj` in train.
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                
Setting param `disable_default_eval_metric` to 1.      
[13:06:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you f

[13:28:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.28547	test-merror:0.29620                            

[1]	train-merror:0.21927	test-merror:0.22710                            

[2]	train-merror:0.20303	test-merror:0.21110                            

[3]	train-merror:0.18883	test-merror:0.19630                            

[4]	train-merror:0.17818	test-merror:0.18900                            

[5]	train-merror:0.16897	test-merror:0.17950                            

[6]	train-merror:0.16488	test-merror:0.17710                            

[7]	train-merror:0.15838	test-merror:0.17090                            

[8]	train-merror:0.15557	test-merror:0.16720         

[1]	train-merror:0.15085	test-merror:0.17810                           

[2]	train-merror:0.14612	test-merror:0.17520                           

[3]	train-merror:0.14333	test-merror:0.17040                           

[4]	train-merror:0.14198	test-merror:0.17070                           

[5]	train-merror:0.14043	test-merror:0.16780                           

[6]	train-merror:0.13883	test-merror:0.16880                           

[7]	train-merror:0.13810	test-merror:0.16680                           

[8]	train-merror:0.13752	test-merror:0.16680                           

[9]	train-merror:0.13702	test-merror:0.16740                           

[10]	train-merror:0.13535	test-merror:0.16760                          

[11]	train-merror:0.13477	test-merror:0.16620                          

[12]	train-merror:0.13483	test-merror:0.16550                          

[13]	train-merror:0.13440	test-merror:0.16520                          

[14]	train-merror:0.13377	test-merror:0.16540      

[6]	train-merror:0.19445	test-merror:0.21000                            

[7]	train-merror:0.18910	test-merror:0.20420                            

[8]	train-merror:0.18283	test-merror:0.20000                            

[9]	train-merror:0.17847	test-merror:0.19470                            

[10]	train-merror:0.17258	test-merror:0.18920                           

[11]	train-merror:0.16772	test-merror:0.18340                           

[12]	train-merror:0.16477	test-merror:0.18110                           

[13]	train-merror:0.16113	test-merror:0.17770                           

[14]	train-merror:0.15810	test-merror:0.17390                           

[15]	train-merror:0.15577	test-merror:0.17270                           

[16]	train-merror:0.15295	test-merror:0.16950                           

[17]	train-merror:0.14983	test-merror:0.16540                           

[18]	train-merror:0.14800	test-merror:0.16410                           

[19]	train-merror:0.14642	test-merror:

[10]	train-merror:0.10347	test-merror:0.14930                            

[11]	train-merror:0.10353	test-merror:0.15010                            

[12]	train-merror:0.10312	test-merror:0.14990                            

[13]	train-merror:0.10317	test-merror:0.14970                            

[14]	train-merror:0.10307	test-merror:0.14920                            

[15]	train-merror:0.10263	test-merror:0.14930                            

[16]	train-merror:0.10248	test-merror:0.14970                            

[17]	train-merror:0.10212	test-merror:0.14910                            

[18]	train-merror:0.10183	test-merror:0.14840                            

[19]	train-merror:0.10128	test-merror:0.14820                            

{'alpha': 0.0009092253399367511, 'btype': 'Rn', 'colsample_bylevel': 0.6272776529200459, 'colsample_bytree': 0.6858001530463793, 'eta': 0.06349290647190063, 'eval_metric': ('merror',), 'extra_dims': 8, 'gamma': 0.5589913986966675, 'lambda': 0.4512344

...

[12]	train-merror:0.05520	test-merror:0.13050                            

[13]	train-merror:0.05405	test-merror:0.12870                            

[14]	train-merror:0.05225	test-merror:0.12850                            

[15]	train-merror:0.05118	test-merror:0.12770                            

[16]	train-merror:0.04967	test-merror:0.12770                            

[17]	train-merror:0.04840	test-merror:0.12680                            

[18]	train-merror:0.04712	test-merror:0.12570                            

[19]	train-merror:0.04590	test-merror:0.12470                            

{'alpha': 0, 'btype': 'R', 'colsample_bylevel': 0.6826183859495948, 'colsample_bytree': 0.9210283048625382, 'eta': 0.12418772502312452, 'eval_metric': ('merror',), 'extra_dims': 3, 'gamma': 0, 'lambda': 0, 'max_depth': 10, 'min_child_weight': 39.89106342536092, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.8442563453821774}
Overwriting param `num_class`                             

[17]	train-merror:0.16628	test-merror:0.17900                            

[18]	train-merror:0.16503	test-merror:0.17760                            

[19]	train-merror:0.16363	test-merror:0.17580                            

{'alpha': 0, 'btype': 'In', 'colsample_bylevel': 0.5882907360977953, 'colsample_bytree': 0.869041192233556, 'eta': 0.08594297043840206, 'eval_metric': ('merror',), 'extra_dims': 5, 'gamma': 0, 'lambda': 0, 'max_depth': 10, 'min_child_weight': 70.83832507426783, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.9478269779914117}
Overwriting param `num_class`                                            
Overwriting param `objective` while setting `obj` in train.              
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                                  
Setting param `disable_default_eval_metric` to 1.                        
[21:47:17] WARNING: /Users/travis/build/dmlc/xg

Overwriting param `num_class`                                            
Overwriting param `objective` while setting `obj` in train.              
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                                  
Setting param `disable_default_eval_metric` to 1.                        
[21:55:25] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.16503	test-merror:0.19100                             

[1]	train-merror:0.15773	test-merror:0.18180                             

[2]	train-merror:0.15397	test-merror:0.18010                             

[3]	train-merror:0.15185	test-merr

[22:02:21] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-merror:0.27905	test-merror:0.28910                             

[1]	train-merror:0.25578	test-merror:0.26690                             

[2]	train-merror:0.24310	test-merror:0.26010                             

[3]	train-merror:0.23543	test-merror:0.24890                             

[4]	train-merror:0.22590	test-merror:0.23950                             

[5]	train-merror:0.21977	test-merror:0.23390                             

[6]	train-merror:0.21512	test-merror:0.22900                             

[7]	train-merror:0.21065	test-merror:0.22420                             

[8]	train-merror:0.20580	test-merror:0.21780 

[0]	train-merror:0.12305	test-merror:0.15260                             

[1]	train-merror:0.11755	test-merror:0.14900                             

[2]	train-merror:0.11557	test-merror:0.14780                             

[3]	train-merror:0.11277	test-merror:0.14600                             

[4]	train-merror:0.11115	test-merror:0.14630                             

[5]	train-merror:0.10952	test-merror:0.14500                             

[6]	train-merror:0.10823	test-merror:0.14480                             

[7]	train-merror:0.10658	test-merror:0.14390                             

[8]	train-merror:0.10465	test-merror:0.14200                             

[9]	train-merror:0.10303	test-merror:0.13960                             

[10]	train-merror:0.10165	test-merror:0.13920                            

[11]	train-merror:0.09998	test-merror:0.13850                            

[12]	train-merror:0.09823	test-merror:0.13810                            

[13]	train-merror:0.09700

In [4]:
print(best_val)
print(space_eval(spc, best))

0.1026
{'alpha': 0, 'btype': 'Rn', 'colsample_bylevel': 0.6822869563299857, 'colsample_bytree': 0.8404138781896768, 'eta': 0.05918034203383484, 'eval_metric': ('merror',), 'extra_dims': 14, 'gamma': 0, 'lambda': 0, 'max_depth': 10, 'min_child_weight': 1.063006375746526, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.8802184601461688}
